<a href="https://colab.research.google.com/github/arkalim/Tensorflow-/blob/master/VLocnet_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import datetime, os

_BATCH_NORM_DECAY = 0.997
_BATCH_NORM_EPSILON = 1e-5

################################################################################
# Helper Functions
################################################################################

# Performs a batch normalization using a standard set of parameters.
def batch_norm(inputs, training, name = 'batch_norm'):
    with tf.name_scope(name):
        # here axis = 3 (channel last format)
        return tf.layers.batch_normalization(
            inputs=inputs, axis=3,
            momentum=_BATCH_NORM_DECAY, epsilon=_BATCH_NORM_EPSILON, center=True,
            scale=True, training=training, fused=True, reuse = False)

def maxpool_2d(inputs, kernel = 2, stride = 2, name = 'maxpool_2d'):
    with tf.name_scope(name):
        return tf.nn.max_pool(inputs, ksize=[1, kernel, kernel, 1], strides=[1, stride, stride, 1], padding='SAME')


def fixed_padding(inputs, kernel_size, name = 'fixed_padding'):
    """Pads the input along the spatial dimensions independently of input size.
    Args:
        inputs: A tensor of size [batch, height_in, width_in, channels]
        
        kernel_size: The kernel size for conv2d or max_pool2d operation
                 
    Returns:
        A tensor with the same format as the input with the data either intact
        (if kernel_size == 1) or padded (if kernel_size > 1)
    """
    with tf.name_scope(name):
        
        pad_total = kernel_size - 1
        pad_beg = pad_total // 2
        pad_end = pad_total - pad_beg

        padded_inputs = tf.pad(tensor=inputs, paddings=[[0, 0], [pad_beg, pad_end], [pad_beg, pad_end], [0, 0]])
        return padded_inputs

def conv2d_fixed_padding(inputs, filters, kernel_size, strides, name = 'conv2d_fp'):
    
    with tf.name_scope(name):
        # inputs will be of shape [-1,row,col,filters_in]
        filters_in = int(inputs.shape[3])

        # if stride > 1 then pad the input with the kernel size to obtain fixed padding
        if strides > 1:
            inputs = fixed_padding(inputs, kernel_size)
            
        with tf.variable_scope('weights', reuse=True):    
            w = tf.Variable(tf.truncated_normal([kernel_size, kernel_size, filters_in, filters], stddev=0.1), name="W")

        output = tf.nn.conv2d(inputs, w, strides=[1, strides, strides, 1], padding=('SAME' if strides == 1 else 'VALID'))  

        return output  

################################################################################
# ResNet block definitions.
################################################################################
def bottleneck_block(inputs, filters, training, projection_shortcut, strides, name = 'bottleneck'):
    """A single block for ResNet v1, with a bottleneck.
    Similar to _building_block_v1(), except using the "bottleneck" blocks.
    Convolution then batch normalization then ReLU.
      
    Args:
        inputs: A tensor of size [batch, height_in, width_in, channels]
        
        filters: The number of filters for the convolutions.
        
        training: A Boolean for whether the model is in training or inference
                    mode. Needed for batch normalization.
                    
        projection_shortcut: The function to use for projection shortcuts
                        (typically a 1x1 convolution when downsampling the input).
                        
        strides: The block's stride. If greater than 1, this block will ultimately
                 downsample the input.
    Returns:
        The output tensor of the block; shape should match inputs.
    """
    with tf.name_scope(name):
        shortcut = inputs

        if projection_shortcut is not None:
            shortcut = projection_shortcut(inputs)
            shortcut = batch_norm(inputs=shortcut, training=training)

        inputs = conv2d_fixed_padding(inputs = inputs, filters = filters, kernel_size=1, strides=1)
        inputs = batch_norm(inputs, training)
        inputs = tf.nn.relu(inputs)

        inputs = conv2d_fixed_padding(inputs = inputs, filters = filters, kernel_size=3, strides=strides)
        inputs = batch_norm(inputs, training)
        inputs = tf.nn.relu(inputs)

        inputs = conv2d_fixed_padding(inputs = inputs, filters = 4 * filters, kernel_size=1, strides=1)
        inputs = batch_norm(inputs, training)

        inputs += shortcut
        output = tf.nn.relu(inputs)

        return output

def stage(inputs, filters, blocks, strides, training, name = 'stage'):
    """Creates one layer of blocks for the ResNet model.
    Args:
        inputs: A tensor of size [batch, height_in, width_in, channels]
        
        filters: The number of filters for the first convolution of the layer
        
        blocks: The number of blocks contained in the layer.
        
        strides: The stride to use for the first convolution of the layer. If greater than 1, 
                 this layer will ultimately downsample the input.
                 
        training: Boolean, whether we are currently training the model. Needed for batch norm.
        
        name: A string name for the tensor output of the block layer.
 
    Returns:
        The output tensor of the block layer.
    """
    
    with tf.name_scope(name):

        # Bottleneck blocks end with 4 x filters 
        filters_out = filters * 4 

        def projection_shortcut(inputs):
            with tf.name_scope('projection_shortcut'):           
                # here number of filters = filters_out to match the shape of the output
                return conv2d_fixed_padding(inputs = inputs, filters = filters_out, kernel_size = 1, strides = strides)

        # we can make the entire resnet model using bottleneck blocks by selecting block_fn as bottleneck_block

        # Only the first block per block_layer uses projection_shortcut and strides
        inputs = bottleneck_block(inputs, filters, training, projection_shortcut, strides, name = 'block_1')

        for i in range(1, blocks):
            # other blocks in a layer do not use projecting shortcuts and they use stride = 1
            inputs = bottleneck_block(inputs, filters, training, projection_shortcut = None, strides = 1, name = 'block_{}'.format(i+1))

        # Naming the output tensor    
        output = tf.identity(inputs, name + '_out')  

        return output

class resnet(object):
# Base class for building the Resnet Model

    def __init__(self, num_classes, num_filters,kernel_size,conv_stride, first_pool_size, first_pool_stride, block_sizes, block_strides):
        
        """Creates a model for classifying an image.
        Args:

            num_classes: The number of classes used as labels.
      
            num_filters: The number of filters to use for the first block layer of the model. 
                   This number is then doubled for each subsequent block layer
                   
            kernel_size: The kernel size to use for convolution.
      
            conv_stride: stride size for the initial convolutional layer
      
            first_pool_size: Pool size to be used for the first pooling layer.
                       If none, the first pooling layer is skipped.
                       
            first_pool_stride: stride size for the first pooling layer. 
                         Not used if first_pool_size is None.
                         
            block_sizes: A list containing n values, where n is the number of sets of
                   block layers desired. Each value should be the number of blocks in the i-th set.
        
            block_strides: List of integers representing the desired stride size for
                     each of the sets of block layers. Should be same length as block_sizes.
        """
        
        self.num_classes = num_classes
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.conv_stride = conv_stride
        self.first_pool_size = first_pool_size
        self.first_pool_stride = first_pool_stride
        self.block_sizes = block_sizes
        self.block_strides = block_strides

    def __call__(self, inputs, training, name):
        """Add operations to classify a batch of input images.
        Args:
            inputs: A Tensor representing a batch of input images.
            
            training: A boolean. Set to True to add operations required only when
                      training the classifier.
        Returns:
            A logits Tensor with shape [<batch_size>, self.num_classes].
        """

        with tf.name_scope(name):
            
            ######################################### Stage:1 ##########################################
            with tf.name_scope('stage_1'):
            
                inputs = conv2d_fixed_padding(inputs=inputs, filters=self.num_filters, kernel_size=self.kernel_size, strides=self.conv_stride)
                inputs = tf.identity(inputs, 'initial_conv')
                inputs = batch_norm(inputs, training)
                inputs = tf.nn.relu(inputs)

                if self.first_pool_size:
                    inputs = tf.compat.v1.layers.max_pooling2d(inputs=inputs, pool_size=self.first_pool_size, strides=self.first_pool_stride, padding='SAME')
                    inputs = tf.identity(inputs, 'initial_max_pool')
            
            ######################################### Stage:2-n ##########################################
            for i, num_blocks in enumerate(self.block_sizes[:-1]):
                  
                num_filters = self.num_filters * (2**i)
                inputs = stage(inputs=inputs, filters = num_filters, blocks=num_blocks, strides=self.block_strides[i], training = training, name='stage_{}'.format(i + 2))
            
            return inputs
               
            
class vlocnet(object):
    
    def __init__(self, num_classes, num_filters,kernel_size,conv_stride, first_pool_size, first_pool_stride, block_sizes, block_strides):
        
        self.num_classes = num_classes
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.conv_stride = conv_stride
        self.first_pool_size = first_pool_size
        self.first_pool_stride = first_pool_stride
        self.block_sizes = block_sizes
        self.block_strides = block_strides
        
    
        self.current_stream = resnet( self.num_classes, self.num_filters,
               self.kernel_size, self.conv_stride, self.first_pool_size, self.first_pool_stride,
               self.block_sizes, self.block_strides)
    
        self.prev_stream = resnet( self.num_classes, self.num_filters,
               self.kernel_size, self.conv_stride, self.first_pool_size, self.first_pool_stride,
               self.block_sizes, self.block_strides)
        
    def __call__(self, current_frame, prev_frame, training):   
        
        with tf.name_scope('Model'):
    
            x = self.current_stream(current_frame, training, name = 'current_stream')
            y = self.prev_stream(prev_frame, training, name = 'prev_stream')

            con = tf.concat([x,y], axis = 3)

            ################# Stage: 5 ##########################
            num_filters = int(con.shape[3])
            inputs = stage(inputs=con, filters = num_filters, blocks=self.block_sizes[-1], strides=self.block_strides[-1], training = training, name='stage_5')


            with tf.name_scope('Output_Stage'):

                    # axis of pooling as we use channel last
                    axes = [1, 2]
                    
                    # global average pooling 
                    inputs = tf.reduce_mean(input_tensor = inputs, axis=axes, keepdims = True)
                    
                    inputs = tf.squeeze(inputs, axes)
                    
                    inputs = tf.layers.dense(inputs=inputs, units = 1024)
                    
                    position = tf.layers.dense(inputs=inputs, units = 3)
                    orientation = tf.layers.dense(inputs=inputs, units = 4)
                    
                    position = tf.identity(position, 'position')
                    orientation = tf.identity(orientation, 'orientation')
                    
                    return position, orientation 

        

In [0]:
model = vlocnet( num_classes = 10, num_filters = 64,
               kernel_size = 7, conv_stride = 2, first_pool_size = 3, first_pool_stride = 2,
               block_sizes = [3,4,6,3], block_strides =[1,2,2,2])

In [0]:
with tf.name_scope("Placeholders"):    
    # Defining the features and labels placeholders
    current_frame = tf.placeholder(tf.float32 ,shape = [None ,224,224,3] , name='current_frame')
    prev_frame = tf.placeholder(tf.float32 ,shape = [None ,224,224,3] , name='prev_frame')
    
    Y = tf.placeholder(tf.float32,shape = [None , 10], name='Y')
    train = tf.placeholder(tf.bool, name = 'train')
 
# Create the CNN model
# keep_prob is the probability to keep a node during training
position, orientation = model(current_frame,prev_frame, train)
'''
with tf.name_scope("Loss"):    
    # Defining the loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))
    tf.summary.scalar("Loss",loss)

with tf.name_scope("Optimizer"):
    # Defining the optimization function
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)

with tf.name_scope("Accuracy"):    
    # The prediction is correct when Y equals pred
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))

    # Defining the accuracy
    # Type casting the prediction to float value and averaging over the entire set
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("Accuracy",accuracy)

'''
with tf.name_scope("Initializer"):    
    # Defining the variable initialisation function
    init = tf.global_variables_initializer()
    

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dense instead.


In [0]:
!pip install -q tf-nightly-2.0-preview

# Load the TensorBoard notebook extension
%load_ext tensorboard

with tf.Session() as sess:
    
    # Initialise the variables
    sess.run(init)
    
    # Clear any prior data in logs
    !rm -rf 'logs'
    
    # create a train summary writer
    writer = tf.summary.FileWriter('logs' , sess.graph)

    # Load Tensorboard at the two directories
    %tensorboard --logdir 'logs'   